In [12]:
!pip install langchain langchain-openai langchain-community faiss-cpu unstructured

In [13]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [14]:
Gemini

2.5 Pro

PRO



Gemini との会話
以下のコードを2点修正してください

1) 会話ログの読み込みを shared_logs_path 以下の一括のテキストファイルの読み込みから、shared_logs_path以下の キャラクター名.csv を個別に読み込むように変更してください

2) 過去のチャット履歴を踏まえて次の発言を行うように変更してください



import os

import random # ランダムな順番を生成するためにインポート

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_community.document_loaders import DirectoryLoader, TextLoader # TextLoaderを追加

from langchain_community.vectorstores import FAISS

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import create_retrieval_chain

from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain.chains.history_aware_retriever import create_history_aware_retriever

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.messages import HumanMessage, AIMessage

from google.colab import drive, userdata



# ==============================================================================

# 1. Characterクラスの定義（RAG学習ロジックを含む）

# ==============================================================================

class Character:

    """キャラクターのRAGチェーンと情報をカプセル化するクラス"""

    def __init__(self, name: str, private_docs_path: str, shared_logs_path: str, system_prompt_template: str):

        self.name = name

        # drive.mount("/content/drive")

        print(f"キャラクター「{self.name}」を構築中...")

        # private_docs_path = os.path.join("/content/drive/My Drive/",private_docs_path)

        # shared_logs_path = os.path.join("/content/drive/My Drive/",shared_logs_path)

        # --- ドキュメント読み込み処理の改良 ---

        # 1. 個別の知識ファイルを読み込む

        private_loader = DirectoryLoader(private_docs_path, glob="**/*.txt")

        private_docs = private_loader.load()

        print(f"  > 個別知識を {len(private_docs)} 件読み込みました。")



        # 2. 共有の会話ログから、自分に関係のあるものだけを読み込む

        shared_docs = []

        for filename in os.listdir(shared_logs_path):

            log_path = os.path.join(shared_logs_path, filename)

            log_loader = TextLoader(log_path)

            shared_docs.extend(log_loader.load())

        print(f"  > 関連する会話ログを {len(shared_docs)} 件読み込みました。")



        # 3. 全てのドキュメントを結合

        all_docs = private_docs + shared_docs

        print(f"  > 合計 {len(all_docs)} 件のドキュメントで知識ベースを構築します。")

        # --- ここまでが改良箇所 ---



        # LLMとEmbeddingモデル

        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

        embeddings = OpenAIEmbeddings()



        # 分割とベクトル化

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

        split_docs = text_splitter.split_documents(all_docs)

        vectorstore = FAISS.from_documents(split_docs, embeddings)

        retriever = vectorstore.as_retriever()



        # RAGチェーンの構築（変更なし）

        history_aware_prompt = ChatPromptTemplate.from_messages([

            MessagesPlaceholder(variable_name="chat_history"),

            ("user", "{input}"),

            ("user", "上記の発言を踏まえ、関連情報を検索するためのキーワードを生成してください。"),

        ])

        history_aware_retriever = create_history_aware_retriever(llm, retriever, history_aware_prompt)

        answer_prompt = ChatPromptTemplate.from_messages([

            ("system", system_prompt_template),

            MessagesPlaceholder(variable_name="chat_history"),

            ("user", "{input}"),

        ])

        document_chain = create_stuff_documents_chain(llm, answer_prompt)

        self.rag_chain = create_retrieval_chain(history_aware_retriever, document_chain)

        print(f"キャラクター「{self.name}」の構築完了。")



    def speak(self, input_text: str, chat_history: list):

        result = self.rag_chain.invoke({"input": input_text, "chat_history": chat_history})

        return result["answer"]



# ==============================================================================

# 2. DialogueManagerクラスの定義（対話フローを改良）

# ==============================================================================

# ==============================================================================

# 2. DialogueManagerクラスの定義（対話フローを改良）

# ==============================================================================

class DialogueManager:

    """対談の進行を管理するクラス"""

    def __init__(self, characters: list, topics: list):

        self.characters = characters

        self.topics = topics

        self.shared_history = []



    def _introduce_topic(self, topic: str):

        print("\n" + "="*50)

        print(f"【新たなテーマ】: {topic}")

        print("="*50 + "\n")

        self.shared_history.append(HumanMessage(content=f"司会者: それでは、次のテーマ「{topic}」について議論を始めましょう。"))

        # 最初の入力は、全キャラクターへの問いかけとする

        return f"最初の議題として、「{topic}」について、皆さんのご意見をお聞かせください。"



    # 【改善点2】対話フローを「キャッチボール形式」に大幅改善

    def run_discussion(self, turns_per_character: int = 5):

        """

        全テーマにわたる対談を実行する。

        直前の発言者とは別のキャラクターが応答することで、自然な対話を実現する。

        """

        for topic in self.topics:

            current_input = self._introduce_topic(topic)

            last_speaker = None # 直前の発言者を記録する変数



            total_turns = len(self.characters) * turns_per_character

            print(f"今回のテーマでは、合計 {total_turns} 回の発言が予定されています。")

            print("-" * 50 + "\n")



            for turn in range(total_turns):

                # 次の発言者を決定する

                # 直前の発言者がいる場合は、そのキャラクターを除いたリストからランダムに選ぶ

                if last_speaker:

                    possible_speakers = [c for c in self.characters if c.name != last_speaker.name]

                    speaker = random.choice(possible_speakers)

                else:

                    # 最初の発言者は全員からランダムに選ぶ

                    speaker = random.choice(self.characters)



                print(f"--- (ターン{turn + 1}/{total_turns}) {speaker.name}の発言 ---")



                response = speaker.speak(current_input, self.shared_history)



                # 念のため、ここでも文字数制限をかける（プロンプトの指示が最も重要）

                response = response[:200]



                print(response)

                print("-" * 20 + "\n")



                # 共有履歴と次の入力を更新

                self.shared_history.append(AIMessage(content=response, name=speaker.name))

                current_input = response

                last_speaker = speaker # 直前の発言者を更新



# ==============================================================================

# 3. メイン処理

# ==============================================================================

if __name__ == "__main__":

    # 【改善点1】プロンプトに文字数制限の指示を追加

    character_definitions = [

        {"name": "忍者", "private_path": "./忍者/", "shared_path": "./conversation_logs/", "prompt": "あなたはJリーグのサッカーに詳しい名古屋グランパスが好きな「忍者」です。一人称は「拙者」。「～でござる」「ﾆﾝﾆﾝ」が語尾に付くことが多いです。冷静に、どこかユーモラスに名古屋グランパスのことを前向きに語ってください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"},

        {"name": "侍", "private_path": "./侍/", "shared_path": "./conversation_logs/", "prompt": "あなたはJリーグの名古屋グランパスを好きな「侍」です。一人称は「侍」。「～であろう」「～なかろう」といった武士を思わせる言葉遣いで話します。データと論理に基づき、冷静かつ客観的に名古屋グランパスの戦いを分析してください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"},

        {"name": "記者", "private_path": "./記者/", "shared_path": "./conversation_logs/", "prompt": "あなたはJリーグの名古屋グランパスが好きな記者です。「ですます」体で話します。一人称は省略して話すことが多いです。テーマについて、侍や忍者に対して自分の思う疑問を投げかけて議論を進めることに注力してください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"}

    ]



    discussion_topics = [

        "名古屋グランパスの2025年度前半（2025年7月まで）の成績をどう評価するのか？",

        "名古屋グランパスの戦術を改善するとしたらなにが必要か？",

        "名古屋グランパスの2025年度後半(2025年8月から12月)になにを期待するか？"

    ]



    # --- 実行 ---

    characters = [

        Character(

            name=c["name"],

            private_docs_path=c["private_path"],

            shared_logs_path=c["shared_path"],

            system_prompt_template=c["prompt"]

        )

        for c in character_definitions

    ]



    manager = DialogueManager(characters, discussion_topics)

    # 【改善点3】各テーマで、各キャラクターが5回ずつ話すように設定

    manager.run_discussion(turns_per_character=5)




思考プロセスを表示
ご依頼いただいた2点の修正を反映したコードは以下の通りです。

会話ログの読み込み方法の変更:
Characterクラスが初期化される際に、shared_logs_pathから全ファイルを読み込むのではなく、自身のキャラクター名（例: 忍者.csv）に一致するCSVファイルのみを個別に読み込むように修正しました。これにより、各キャラクターは自分に関連する過去の会話ログのみを知識ベースに追加します。

過去のチャット履歴を考慮した発言:
ご提示のコードは、LangChainのcreate_history_aware_retrieverとMessagesPlaceholderを用いることで、すでに過去の会話履歴(chat_history)を考慮した応答生成の仕組みが実装されていました。この優れた設計を維持しつつ、コードの意図がより明確になるように、DialogueManagerクラス内のコメントを充実させ、対話がどのように履歴を考慮して進むのかを説明するドキュメント（docstring）を追加しました。ロジック自体は元の自然な会話フローを維持しています。

修正後のコード
Python

import os
import random
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# ★修正点: CSVLoaderを追加
from langchain_community.document_loaders import DirectoryLoader, TextLoader, CSVLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
# from google.colab import drive, userdata # Google Colab環境以外では不要

# ==============================================================================
# 1. Characterクラスの定義（RAG学習ロジックを含む）
# ==============================================================================
class Character:
    """キャラクターのRAGチェーンと情報をカプセル化するクラス"""
    def __init__(self, name: str, private_docs_path: str, shared_logs_path: str, system_prompt_template: str):
        self.name = name
        print(f"キャラクター「{self.name}」を構築中...")

        # --- ドキュメント読み込み処理の改良 ---
        # 1. 個別の知識ファイル（プライベートな情報）を読み込む
        private_loader = DirectoryLoader(private_docs_path, glob="**/*.txt", loader_cls=TextLoader)
        private_docs = private_loader.load()
        print(f"  > 個別知識を {len(private_docs)} 件読み込みました。")

        # ★修正点1: 共有の会話ログから、自分自身のCSVファイルのみを読み込むように変更
        shared_docs = []
        character_log_file = os.path.join(shared_logs_path, f"{self.name}.csv")

        if os.path.exists(character_log_file):
            try:
                # 自身のキャラクター名のCSVファイルを読み込む
                log_loader = CSVLoader(file_path=character_log_file, encoding='utf-8')
                shared_docs = log_loader.load()
                print(f"  > 共有会話ログ '{os.path.basename(character_log_file)}' を読み込み、{len(shared_docs)} 件のドキュメントを取得しました。")
            except Exception as e:
                print(f"  > 共有会話ログ '{os.path.basename(character_log_file)}' の読み込み中にエラーが発生しました: {e}")
        else:
            print(f"  > 共有会話ログ '{os.path.basename(character_log_file)}' が見つかりませんでした。")

        # 3. 全てのドキュメントを結合
        all_docs = private_docs + shared_docs
        print(f"  > 合計 {len(all_docs)} 件のドキュメントで知識ベースを構築します。")
        # --- ここまでが改良箇所 ---

        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
        embeddings = OpenAIEmbeddings()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        split_docs = text_splitter.split_documents(all_docs)
        vectorstore = FAISS.from_documents(split_docs, embeddings)
        retriever = vectorstore.as_retriever()

        # RAGチェーンの構築（会話履歴を考慮する設計）
        history_aware_prompt = ChatPromptTemplate.from_messages([
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            ("user", "上記の発言を踏まえ、関連情報を検索するためのキーワードを生成してください。"),
        ])
        history_aware_retriever = create_history_aware_retriever(llm, retriever, history_aware_prompt)

        answer_prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt_template),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
        ])
        document_chain = create_stuff_documents_chain(llm, answer_prompt)
        self.rag_chain = create_retrieval_chain(history_aware_retriever, document_chain)
        print(f"キャラクター「{self.name}」の構築完了。")

    def speak(self, input_text: str, chat_history: list):
        """与えられた入力と会話履歴に基づいて発言を生成する"""
        result = self.rag_chain.invoke({"input": input_text, "chat_history": chat_history})
        return result["answer"]

# ==============================================================================
# 2. DialogueManagerクラスの定義（対話フローを改良）
# ==============================================================================
class DialogueManager:
    """対談の進行を管理するクラス"""
    def __init__(self, characters: list, topics: list):
        self.characters = characters
        self.topics = topics
        self.shared_history = []

    def _introduce_topic(self, topic: str):
        """トピックを紹介し、最初の発言を促すメッセージを作成する"""
        print("\n" + "="*50)
        print(f"【新たなテーマ】: {topic}")
        print("="*50 + "\n")
        moderator_message = f"司会者: それでは、次のテーマ「{topic}」について議論を始めましょう。"
        self.shared_history.append(HumanMessage(content=moderator_message))
        return f"最初の議題として、「{topic}」について、皆さんのご意見をお聞かせください。"

    # ★修正点2: 過去のチャット履歴を踏まえた対話フローの明確化
    def run_discussion(self, turns_per_character: int = 5):
        """
        全テーマにわたる対談を実行する。
        共有の会話履歴(self.shared_history)を各キャラクターに渡すことで、
        過去の発言の文脈を踏まえた応答を生成させる。
        """
        for topic in self.topics:
            current_input = self._introduce_topic(topic)
            last_speaker = None

            total_turns = len(self.characters) * turns_per_character
            print(f"今回のテーマでは、合計 {total_turns} 回の発言が予定されています。")
            print("-" * 50 + "\n")

            for turn in range(total_turns):
                # 1. 次の発言者を決定
                if last_speaker:
                    possible_speakers = [c for c in self.characters if c.name != last_speaker.name]
                    speaker = random.choice(possible_speakers)
                else:
                    speaker = random.choice(self.characters)

                print(f"--- (ターン{turn + 1}/{total_turns}) {speaker.name}の発言 ---")

                # 2. 発言を生成
                # Character.speakに直前の発言(current_input)と全会話履歴(self.shared_history)を渡す。
                # Character内のチェーンが、これらの情報から文脈を理解して応答を生成する。
                response = speaker.speak(current_input, self.shared_history)
                response = response[:200]

                print(response)
                print("-" * 20 + "\n")

                # 3. 共有履歴を更新
                # 今回の発言を、発言者(name)を明記して履歴に追加。
                # これにより、次の発言者はこの発言もコンテキストとして利用できる。
                self.shared_history.append(AIMessage(content=response, name=speaker.name))

                # 4. 次の入力と発言者を更新
                # 次のキャラクターへの入力を、今回の発言内容そのものに設定し、
                # 自然な会話のキャッチボールを実現する。
                current_input = response
                last_speaker = speaker

# ==============================================================================
# 3. メイン処理
# ==============================================================================
if __name__ == "__main__":
    character_definitions = [
        {"name": "忍者", "private_path": "./忍者/", "shared_path": "./conversation_logs/", "prompt": "あなたはJリーグのサッカーに詳しい名古屋グランパスが好きな「忍者」です。一人称は「拙者」。「～でござる」「ﾆﾝﾆﾝ」が語尾に付くことが多いです。冷静に、どこかユーモラスに名古屋グランパスのことを前向きに語ってください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"},
        {"name": "侍", "private_path": "./侍/", "shared_path": "./conversation_logs/", "prompt": "あなたはJリーグの名古屋グランパスを好きな「侍」です。一人称は「侍」。「～であろう」「～なかろう」といった武士を思わせる言葉遣いで話します。データと論理に基づき、冷静かつ客観的に名古屋グランパスの戦いを分析してください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"},
        {"name": "記者", "private_path": "./記者/", "shared_path": "./conversation_logs/", "prompt": "あなたはJリーグの名古屋グランパスが好きな記者です。「ですます」体で話します。一人称は省略して話すことが多いです。テーマについて、侍や忍者に対して自分の思う疑問を投げかけて議論を進めることに注力してください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"}
    ]

    discussion_topics = [
        "名古屋グランパスの2025年度前半（2025年7月まで）の成績をどう評価するのか？",
        "名古屋グランパスの戦術を改善するとしたらなにが必要か？",
        "名古屋グランパスの2025年度後半(2025年8月から12月)になにを期待するか？"
    ]

    # --- 実行 ---
    print("対談シミュレーションを開始します...")
    characters = [
        Character(
            name=c["name"],
            private_docs_path=c["private_path"],
            shared_logs_path=c["shared_path"],
            system_prompt_template=c["prompt"]
        )
        for c in character_definitions
    ]

    manager = DialogueManager(characters, discussion_topics)
    manager.run_discussion(turns_per_character=5)
    print("\n対談シミュレーションを終了します。")





上記のコードをGoogle ColabからGitHub上に保存して、読み込むファイルをGitHub上に配置して実行したところ、File not Foundになりました。



FileNotFoundError                         Traceback (most recent call last)

/tmp/ipython-input-415194212.py in <cell line: 0>()

    161     # --- 実行 ---

    162     print("対談シミュレーションを開始します...")

--> 163     characters = [

    164         Character(

    165             name=c["name"],



3 frames

/usr/local/lib/python3.11/dist-packages/langchain_community/document_loaders/directory.py in lazy_load(self)

    121         p = Path(self.path)

    122         if not p.exists():

--> 123             raise FileNotFoundError(f"Directory not found: '{self.path}'")

    124         if not p.is_dir():

    125             raise ValueError(f"Expected directory, got file: '{self.path}'")



FileNotFoundError: Directory not found: './忍者/'



その理由の指摘と改善案を出してください。




思考プロセスを表示
はい、承知いたしました。FileNotFoundErrorが発生した原因と、その改善案について解説します。

エラーの原因
エラーメッセージ FileNotFoundError: Directory not found: './忍者/' は、プログラムを実行している場所（カレントディレクトリ）に '忍者' という名前のディレクトリが見つからないことを示しています。

Google ColabからGitHubに環境を移行した場合、このようなエラーが起こる主な理由は、ファイルの配置場所の考え方が異なるためです。

Google Colabの環境:
Colabでは、多くの場合 /content/ という一時的な作業ディレクトリに、忍者や侍といったフォルダを直接アップロードしたり作成したりして作業します。プログラム内の './忍者/' という相対パスは、この/content/忍者/ を指していました。

GitHubリポジトリの環境:
GitHubでコードを管理する場合、Pythonスクリプト（.pyファイルや.ipynbファイル）だけでなく、プログラムが読み込むデータファイルやディレクトリ（忍者/, 侍/, conversation_logs/ など）も全てリポジトリに含め、一緒にgit pushする必要があります。

今回のエラーは、PythonスクリプトはGitHubにあるものの、それが参照しようとしている./忍者/ディレクトリがリポジトリ内に存在しないため発生しています。

改善案
対策として、リポジトリのファイル構成を見直す方法と、コードのパス指定をより堅牢にする方法の2つを提案します。

【対策1】ディレクトリとファイルをGitHubリポジトリに追加する
最も直接的な解決策は、ローカル環境で正しいディレクトリ構造を作成し、それをGitHubにプッシュすることです。Pythonスクリプトと同じ階層に、必要なディレクトリとファイルを配置してください。

理想的なディレクトリ構造の例:

your-github-repository/
├── main.py                     <-- 実行するPythonスクリプト
├── 忍者/
│   └── knowledge.txt           <-- 忍者の知識ファイル
├── 侍/
│   └── knowledge.txt           <-- 侍の知識ファイル
├── 記者/
│   └── knowledge.txt           <-- 記者の知識ファイル
└── conversation_logs/
    ├── 忍者.csv
    ├── 侍.csv
    └── 記者.csv
この構造を作成したら、以下のgitコマンドでリポジトリに追加してプッシュします。

Bash

# 変更されたファイルと新しいファイルをすべてステージング
git add .

# コミットする
git commit -m "Add character data directories and files"

# GitHubにプッシュする
git push
【対策2】コードを修正してパスの指定を堅牢にする（推奨）
どこからスクリプトを実行してもエラーが起きないように、パスの指定方法をより信頼性の高いものに修正することをお勧めします。スクリプト自身の場所を基準に、他のファイルへの絶対パスを生成する方法です。

以下のように、メイン処理（if __name__ == "__main__":ブロック）の部分を修正してください。

Python

# ==============================================================================
# 3. メイン処理
# ==============================================================================
if __name__ == "__main__":
    # --- ★ここから修正★ ---
    # スクリプト自身のディレクトリの絶対パスを取得
    script_dir = os.path.dirname(os.path.abspath(__file__))

    character_definitions = [
        {
            "name": "忍者",
            # スクリプトの場所を基準にした絶対パスに変換
            "private_path": os.path.join(script_dir, "忍者/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグのサッカーに詳しい名古屋グランパスが好きな「忍者」です。一人称は「拙者」。「～でござる」「ﾆﾝﾆﾝ」が語尾に付くことが多いです。冷静に、どこかユーモラスに名古屋グランパスのことを前向きに語ってください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"
        },
        {
            "name": "侍",
            "private_path": os.path.join(script_dir, "侍/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグの名古屋グランパスを好きな「侍」です。一人称は「侍」。「～であろう」「～なかろう」といった武士を思わせる言葉遣いで話します。データと論理に基づき、冷静かつ客観的に名古屋グランパスの戦いを分析してください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"
        },
        {
            "name": "記者",
            "private_path": os.path.join(script_dir, "記者/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグの名古屋グランパスが好きな記者です。「ですます」体で話します。一人称は省略して話すことが多いです。テーマについて、侍や忍者に対して自分の思う疑問を投げかけて議論を進めることに注力してください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"
        }
    ]
    # --- ★ここまで修正★ ---

    discussion_topics = [
        "名古屋グランパスの2025年度前半（2025年7月まで）の成績をどう評価するのか？",
        "名古屋グランパスの戦術を改善するとしたらなにが必要か？",
        "名古屋グランパスの2025年度後半(2025年8月から12月)になにを期待するか？"
    ]

    # --- 実行 ---
    print("対談シミュレーションを開始します...")
    characters = [
        Character(
            name=c["name"],
            private_docs_path=c["private_path"],
            shared_logs_path=c["shared_path"],
            system_prompt_template=c["prompt"]
        )
        for c in character_definitions
    ]

    manager = DialogueManager(characters, discussion_topics)
    manager.run_discussion(turns_per_character=5)
    print("\n対談シミュレーションを終了します。")
この修正のポイント:

os.path.dirname(os.path.abspath(__file__)) を使って、実行しているスクリプトファイルがあるディレクトリの絶対パスを取得します。

os.path.join() を使って、その絶対パスと相対パス（例: "忍者/"）を安全に結合し、環境に依存しない正確なパスを生成します。

まず【対策1】でファイル構成を整え、その上で【対策2】のコード修正を行うことで、より安定して動作するプログラムになります。






上記の修正を施したところ、以下のエラーが表示されました

原因と修正案を提示してください

NameError                                 Traceback (most recent call last)

/tmp/ipython-input-3547646033.py in <cell line: 0>()

    147 if __name__ == "__main__":

    148     # スクリプト自身のディレクトリの絶対パスを取得

--> 149     script_dir = os.path.dirname(os.path.abspath(__file__))

    150

    151     character_definitions = [



NameError: name '__file__' is not defined




思考プロセスを表示
はい、承知いたしました。表示されたNameErrorの原因と修正案を提示します。

原因
NameError: name '__file__' is not definedというエラーは、__file__という特別な変数が定義されていないために発生します。

__file__は、実行中のPythonスクリプトファイル（.pyファイル）のパスを格納する変数です。しかし、Google ColabやJupyter Notebookのような**インタラクティブなノートブック環境では、コードをセル単位で実行するため「スクリプトファイル」という概念がありません。**そのため、__file__変数が存在せず、このエラーが発生します。

前回提示した修正案は、通常のPythonスクリプトファイル（.py）で実行することを想定したものでした。引き続きノートブック環境で実行される場合には、別の方法でパスを取得する必要があります。

修正案
ノートブック環境で現在の作業ディレクトリのパスを取得するには、__file__の代わりにos.getcwd()を使用するのが最も簡単で確実です。getcwdは "get current working directory" の略です。

以下のように、メイン処理（if __name__ == "__main__":ブロック）の該当箇所を1行だけ修正してください。

Python

# ==============================================================================
# 3. メイン処理
# ==============================================================================
if __name__ == "__main__":
    # --- ★ここから修正★ ---
    # スクリプト自身のディレクトリの絶対パスを取得
    # ノートブック環境では__file__が使えないため、os.getcwd()で現在の作業ディレクトリを取得する
    script_dir = os.getcwd()
    # --- ★ここまで修正★ ---

    character_definitions = [
        {
            "name": "忍者",
            # スクリプトの場所を基準にした絶対パスに変換
            "private_path": os.path.join(script_dir, "忍者/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグのサッカーに詳しい名古屋グランパスが好きな「忍者」です。一人称は「拙者」。「～でござる」「ﾆﾝﾆﾝ」が語尾に付くことが多いです。冷静に、どこかユーモラスに名古屋グランパスのことを前向きに語ってください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"
        },
        {
            "name": "侍",
            "private_path": os.path.join(script_dir, "侍/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグの名古屋グランパスを好きな「侍」です。一人称は「侍」。「～であろう」「～なかろう」といった武士を思わせる言葉遣いで話します。データと論理に基づき、冷静かつ客観的に名古屋グランパスの戦いを分析してください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"
        },
        {
            "name": "記者",
            "private_path": os.path.join(script_dir, "記者/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグの名古屋グランパスが好きな記者です。「ですます」体で話します。一人称は省略して話すことが多いです。テーマについて、侍や忍者に対して自分の思う疑問を投げかけて議論を進めることに注力してください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"
        }
    ]

    discussion_topics = [
        "名古屋グランパスの2025年度前半（2025年7月まで）の成績をどう評価するのか？",
        "名古屋グランパスの戦術を改善するとしたらなにが必要か？",
        "名古屋グランパスの2025年度後半(2025年8月から12月)になにを期待するか？"
    ]

    # --- 実行 ---
    # (以下、変更なし)
    print("対談シミュレーションを開始します...")
    characters = [
        Character(
            name=c["name"],
            private_docs_path=c["private_path"],
            shared_logs_path=c["shared_path"],
            system_prompt_template=c["prompt"]
        )
        for c in character_definitions
    ]

    manager = DialogueManager(characters, discussion_topics)
    manager.run_discussion(turns_per_character=5)
    print("\n対談シミュレーションを終了します。")
この修正により、ノートブックのファイル（.ipynb）が置かれているディレクトリを基準として、忍者/やconversation_logs/といったデータフォルダへのパスが正しく生成されるようになります。






上記のコードはGitHub上のノートブックに保存されており、それを実行したときにGitHub上のノートブックから見たファイルが読み込めないようです。

対策はありますか？




思考プロセスを表示
はい、承知いたしました。その問題はGitHubと実行環境の関係でよく起こるものです。

端的に言うと、GitHubのサイト上で直接ノートブック（.ipynbファイル）を実行して、リポジトリ内の他のファイル（.csvや.txt）を読み込むことはできません。

## なぜ読み込めないのか？（原因）
GitHubのウェブサイトでノートブックファイルを開くと、表示されているのはコードの実行結果を静的に表示したプレビューです。これは、プログラムが動いているのではなく、いわば「完成形のスクリーンショット」を見ている状態です。

そのため、os.getcwd() のようなファイル操作のコードを実行しようとしても、実際にはPythonのプログラムを動かすための「エンジン」（実行環境）が存在しないため、ファイルを見つけられずエラーになります。

GitHubはあくまで**コードを保存・共有するための場所（倉庫）**であり、コードを実行するための場所（作業場）ではない、と考えると分かりやすいです。

## どうすれば実行できるのか？（対策）
GitHubリポジトリにあるノートブックとファイルを一緒に実行するには、「実行環境」にリポジトリの全ファイルをコピー（クローン）してくる必要があります。

代表的で簡単な方法を2つご紹介します。

【対策1】Google Colab を利用する方法（一番おすすめ）
ユーザー様が慣れているGoogle Colabを使い、GitHubリポジトリを丸ごと読み込んで実行する方法です。

Google Colabを開き、GitHubからノートブックを開く

Colabのメニューから ファイル > ノートブックを開く を選択します。

表示されたウィンドウで GitHub タブを選び、あなたのGitHubリポジトリのURLを貼り付けて、実行したいノートブックファイル（.ipynb）を選択します。

リポジトリ全体をColab環境にクローンする

ノートブックが開いたら、最初のセルで以下のコマンドを実行し、データファイルを含むリポジトリ全体をColabにダウンロードします。

your-username と your-repository-name はご自身のものに置き換えてください。

Python

# !git clone を使って、GitHubリポジトリをColabに丸ごとコピーします
!git clone https://github.com/your-username/your-repository-name.git
作業ディレクトリを移動する

クローンすると、リポジトリ名のついたディレクトリが作成されます。その中に移動しないと、ノートブックはファイルを見つけられません。次のセルで以下のコードを実行してください。

Python

import os
# 作成されたリポジトリのディレクトリに移動する
os.chdir('your-repository-name')
ノートブックの残りのセルを実行する

これ以降は、以前のコード script_dir = os.getcwd() が正しく動作し、忍者/ などのディレクトリを正常に読み込めるようになります。

【対策2】自分のPC上で実行する方法（ローカル環境）
ご自身のPCにPythonやJupyter Notebookの環境を構築して実行する方法です。

Gitをインストールする

もしPCにGitがなければ、インストールしてください。

リポジトリをPCにクローンする

コマンドプロンプトやターミナルを開き、作業したい場所に移動してから以下のコマンドを実行します。

Bash

git clone https://github.com/your-username/your-repository-name.git
Jupyter Notebookを起動する

クローンして作成されたディレクトリに移動し、Jupyter Notebookを起動します。

Bash

# ディレクトリに移動
cd your-repository-name

# Jupyter Notebookを起動
jupyter notebook
ノートブックを開いて実行

ブラウザが自動で開き、ファイル一覧が表示されます。そこからノートブックファイルをクリックして開き、実行します。この方法なら、ファイルはすべて同じ場所にあるため、問題なく読み込めます。



import os
import random
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader, CSVLoader, WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# ==============================================================================
# 1. Characterクラスの定義（RAG学習ロジックを含む）
# ==============================================================================
class Character:
    """キャラクターのRAGチェーンと情報をカプセル化するクラス"""
    # ★追加★ web_urls引数を追加
    def __init__(self, name: str, private_docs_path: str, shared_logs_path: str, system_prompt_template: str, web_urls: list):
        self.name = name
        print(f"キャラクター「{self.name}」を構築中...")

        # --- ドキュメント読み込み処理の改良 ---
        # 1. 個別の知識ファイル（プライベートな情報）を読み込む
        private_loader = DirectoryLoader(private_docs_path, glob="**/*.txt", loader_cls=TextLoader)
        private_docs = private_loader.load()
        print(f"  > 個別知識を {len(private_docs)} 件読み込みました。")

        # 2. 共有の会話ログから、自分自身のCSVファイルのみを読み込む
        shared_docs = []
        character_log_file = os.path.join(shared_logs_path, f"{self.name}.csv")
        if os.path.exists(character_log_file):
            try:
                log_loader = CSVLoader(file_path=character_log_file, encoding='utf-8')
                shared_docs = log_loader.load()
                print(f"  > 共有会話ログ '{os.path.basename(character_log_file)}' を読み込み、{len(shared_docs)} 件のドキュメントを取得しました。")
            except Exception as e:
                print(f"  > 共有会話ログの読み込み中にエラーが発生しました: {e}")

        # ★追加★ 3. Webサイトからファクト情報をリアルタイムで読み込む
        web_docs = []
        if web_urls:
            print(f"  > Webサイトから最新のファクト情報を読み込みます...")
            try:
                loader = WebBaseLoader(web_urls)
                web_docs = loader.load()
                print(f"  > Webサイトから {len(web_docs)} 件のドキュメントを取得しました。")
            except Exception as e:
                print(f"  > Webサイトの読み込み中にエラーが発生しました: {e}")

        # ★修正★ 4. 全てのドキュメントを結合（個別知識 + 会話ログ + Web情報）
        all_docs = private_docs + shared_docs + web_docs
        print(f"  > 合計 {len(all_docs)} 件のドキュメントで知識ベースを構築します。")
        # --- ここまでが改良箇所 ---

        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
        embeddings = OpenAIEmbeddings()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        split_docs = text_splitter.split_documents(all_docs)
        vectorstore = FAISS.from_documents(split_docs, embeddings)
        retriever = vectorstore.as_retriever()

        history_aware_prompt = ChatPromptTemplate.from_messages([
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            ("user", "上記の発言を踏まえ、関連情報を検索するためのキーワードを生成してください。"),
        ])
        history_aware_retriever = create_history_aware_retriever(llm, retriever, history_aware_prompt)

        answer_prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt_template),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
        ])
        document_chain = create_stuff_documents_chain(llm, answer_prompt)
        self.rag_chain = create_retrieval_chain(history_aware_retriever, document_chain)
        print(f"キャラクター「{self.name}」の構築完了。")

    def speak(self, input_text: str, chat_history: list):
        result = self.rag_chain.invoke({"input": input_text, "chat_history": chat_history})
        return result["answer"]

# ==============================================================================
# 2. DialogueManagerクラスの定義（変更なし）
# ==============================================================================
class DialogueManager:
    """対談の進行を管理するクラス"""
    def __init__(self, characters: list, topics: list):
        self.characters = characters
        self.topics = topics
        self.shared_history = []

    def _introduce_topic(self, topic: str):
        print("\n" + "="*50)
        print(f"【新たなテーマ】: {topic}")
        print("="*50 + "\n")
        moderator_message = f"司会者: それでは、次のテーマ「{topic}」について議論を始めましょう。"
        self.shared_history.append(HumanMessage(content=moderator_message))
        return f"最初の議題として、「{topic}」について、皆さんのご意見をお聞かせください。"

    def run_discussion(self, turns_per_character: int = 5):
        for topic in self.topics:
            current_input = self._introduce_topic(topic)
            last_speaker = None
            total_turns = len(self.characters) * turns_per_character
            print(f"今回のテーマでは、合計 {total_turns} 回の発言が予定されています。")
            print("-" * 50 + "\n")
            for turn in range(total_turns):
                if last_speaker:
                    possible_speakers = [c for c in self.characters if c.name != last_speaker.name]
                    speaker = random.choice(possible_speakers)
                else:
                    speaker = random.choice(self.characters)
                print(f"--- (ターン{turn + 1}/{total_turns}) {speaker.name}の発言 ---")
                response = speaker.speak(current_input, self.shared_history)
                response = response[:200]
                print(response)
                print("-" * 20 + "\n")
                self.shared_history.append(AIMessage(content=response, name=speaker.name))
                current_input = response
                last_speaker = speaker

# ==============================================================================
# 3. メイン処理
# ==============================================================================
if __name__ == "__main__":
    # ノートブック環境などではこちらを使用
    script_dir = os.getcwd()

    # ★追加★ 参照するWebサイトのURL
    fact_urls = [
        "https://www.football-lab.jp/nago/",       # 名古屋グランパスのスタッツ
        "https://www.jleague.jp/standings/j1/"  # J1リーグ順位表
    ]

    # ★修正★ プロンプトを更新し、Webサイトの情報を活用するように指示
    character_definitions = [
        {
            "name": "忍者",
            "private_path": os.path.join(script_dir, "忍者/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグのサッカーに詳しい名古屋グランパスが好きな「忍者」です。一人称は「拙者」。「～でござる」「ﾆﾝﾆﾝ」が語尾に付くことが多いです。\n**重要: 提供されたWebサイトの情報（最新の順位表やチームのスタッツ）を積極的に利用して、具体的でデータに基づいた発言をしてください。**\nあなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。\n{context}"
        },
        {
            "name": "侍",
            "private_path": os.path.join(script_dir, "侍/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグの名古屋グランパスを好きな「侍」です。一人称は「侍」。「～であろう」「～なかろう」といった武士を思わせる言葉遣いで話します。\n**重要: 提供されたWebサイトの情報（最新の順位表やチームのスタッツ）を積極的に利用して、具体的でデータに基づいた発言をしてください。**\nあなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。\n{context}"
        },
        {
            "name": "記者",
            "private_path": os.path.join(script_dir, "記者/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグの名古屋グランパスが好きな記者です。「ですます」体で話します。テーマについて、Webサイトから得られる客観的なデータ（順位、スタッツ等）を基に、侍や忍者に対して鋭い質問を投げかけて議論を深めてください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"
        }
    ]

    # 今日の日付に合わせて、より具体的なテーマに更新
    today_str = "2025年7月31日"
    discussion_topics = [
        f"{today_str}時点での名古屋グランパスの成績（順位、スタッツ）をどう評価するか？",
        "現在のチームの強みと弱点をデータからどう分析するか？",
        "シーズン後半戦、順位を上げるために何が必要か？"
    ]

    # --- 実行 ---
    print("対談シミュレーションを開始します...")
    characters = [
        Character(
            name=c["name"],
            private_docs_path=c["private_path"],
            shared_logs_path=c["shared_path"],
            system_prompt_template=c["prompt"],
            web_urls=fact_urls  # ★追加★ WebサイトのURLを渡す
        )
        for c in character_definitions
    ]

    manager = DialogueManager(characters, discussion_topics)
    manager.run_discussion(turns_per_character=5)
    print("\n対談シミュレーションを終了します。")

対談シミュレーションを開始します...
キャラクター「忍者」を構築中...
  > 個別知識を 1 件読み込みました。
  > 共有会話ログ '忍者.csv' を読み込み、174 件のドキュメントを取得しました。
  > 合計 175 件のドキュメントで知識ベースを構築します。
キャラクター「忍者」の構築完了。
キャラクター「侍」を構築中...
  > 個別知識を 1 件読み込みました。
  > 共有会話ログ '侍.csv' を読み込み、62 件のドキュメントを取得しました。
  > 合計 63 件のドキュメントで知識ベースを構築します。
キャラクター「侍」の構築完了。
キャラクター「記者」を構築中...
  > 個別知識を 1 件読み込みました。
  > 共有会話ログ '記者.csv' を読み込み、228 件のドキュメントを取得しました。
  > 合計 229 件のドキュメントで知識ベースを構築します。
キャラクター「記者」の構築完了。

【新たなテーマ】: 名古屋グランパスの2025年度前半（2025年7月まで）の成績をどう評価するのか？

今回のテーマでは、合計 15 回の発言が予定されています。
--------------------------------------------------

--- (ターン1/15) 侍の発言 ---
侍: 侍
「成敗！（Say-Bye！）」: 「2025年度前半の名古屋グランパスの成績は、戦術の浸透度や選手のコンディションに大きく左右されるであろう。特に、守備の安定性が欠如していた場合、失点が増え、順位に影響を及ぼすことが予想されるなか、攻撃陣の連携が改善されれば、勝利を重ねる可能性も高まるであろう。冷静にデータを分析し、戦術を見直すことが肝要であるな。」成敗！（Say-Bye！）
--------------------

--- (ターン2/15) 忍者の発言 ---
忍者: 忍者
「成敗！（Say-Bye！）」: 「2025年度前半の名古屋グランパス、評価は難しいでござるが、攻撃陣の連携が鍵を握るでござるな。守備が安定せず失点が多ければ、順位が下がることも。また、新戦力の活躍が期待される中、選手のコンディションも重要でござる。冷静にデータを分析し、戦術を見直すことが、さらなる飛躍へと繋がるはずでござるよ。ﾆﾝ

KeyboardInterrupt: 

In [ ]:
!git clone https://github.com/YukinobuYoshihara/yawarakame.git

In [ ]:
import os
# 作成されたリポジトリのディレクトリに移動する
os.chdir('yawarakame')